# Learn Trading Strategy - Collect data

The goal of the project is to implement test strategy to trade gas/oil companies based on the price of gas/oil.

In [ ]:
!pip install yfinance

In [ ]:
import time
from datetime import date

import numpy as np
import pandas as pd
import pandas_datareader as pdr
import plotly.express as px
import plotly.graph_objs as go
import yfinance as yf


In [ ]:
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
# Update (apr 2023): Colab is now Python 3.10
# there are some other (older) ways to install talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3971    0  3971    0     0   7765      0 --:--:-- --:--:-- --:--:--  7771
100  517k  100  517k    0     0   351k      0  0:00:01  0:00:01 --:--:--  612k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4019    0  4019    0     0   7824      0 --:--:-- --:--:-- --:--:--  7834
100  392k  100  392k    0     0   287k      0  0:00:01  0:00:01 --:--:--  688k


In [ ]:
# supress warnings
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 7; day=8
Period for indexes: 1954-07-08 to 2024-07-08 


## Common functions

In [ ]:
def get_growth_df(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """Get growth metrics (close price vs close price fro some previous day) from a dataframe
    """
    for i in [1,3,7,30,90,365]:
        df['growth_'+prefix+'_'+str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
        GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
    return df[GROWTH_KEYS]

In [ ]:
def plot_ts(ts: pd.Series, ylabel: str = "Adj Close", title: str = 'Value over time') -> None:
    # Dynamic visualisation of prices
    fig = px.line(
        ts,
        x=ts.index,
        y=ylabel,
        title=title
    )
    fig.update_layout(title_x=0.5)  # This will center the title horizontally

    fig.show()

## Macro indicators

In [ ]:
# DAX daily
dax_daily = yf.download(
    tickers = "^GDAXI",
    period = "max",
    interval = "1d"
)

dax_daily.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,18474.269531,18541.140625,18444.060547,18531.859375,18531.859375,0


In [ ]:
# add growth metrics
dax_daily_to_merge = get_growth_df(dax_daily, 'dax')
dax_daily_to_merge.tail(2)

,growth_dax_1d,growth_dax_3d,growth_dax_7d,growth_dax_30d,growth_dax_90d,growth_dax_365d
Date,,,,,,
2024-07-05,1.001353,1.017143,1.017637,0.988342,1.052343,1.221430
2024-07-08,1.003053,1.008562,1.017644,0.987065,1.052874,1.224982


In [ ]:
# SNP500

# https://finance.yahoo.com/quote/%5EGSPC/
# SNP - SNP Real Time Price. Currency in USD
snp500_daily = yf.download(
    tickers = "^GSPC",
    period = "max",
    interval = "1d"
)

snp500_daily.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-05,5537.910156,5570.330078,5531.629883,5567.189941,5567.189941,3253080000


In [ ]:
snp500_to_merge = get_growth_df(snp500_daily, 'snp500')
snp500_to_merge.tail(2)

,growth_snp500_1d,growth_snp500_3d,growth_snp500_7d,growth_snp500_30d,growth_snp500_90d,growth_snp500_365d
Date,,,,,,
2024-07-03,1.005084,1.014017,1.016364,1.043121,1.088080,1.420167
2024-07-05,1.005449,1.016822,1.017898,1.046187,1.098167,1.401393


In [ ]:
# Dow Jones Industrial Average: https://finance.yahoo.com/quote/%5EDJI?.tsrc=fin-srch
dji_daily = yf.download(
    tickers = "^DJI",
    period = "max",
    interval = "1d"
)

dji_daily.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-05,39313.398438,39399.621094,39168.699219,39375.871094,39375.871094,325230000


In [ ]:
dji_daily_to_merge = get_growth_df(dji_daily, 'dji')
dji_daily_to_merge.tail(2)

,growth_dji_1d,growth_dji_3d,growth_dji_7d,growth_dji_30d,growth_dji_90d,growth_dji_365d
Date,,,,,,
2024-07-03,0.999394,1.004835,0.997381,0.987470,1.004510,1.189545
2024-07-05,1.001727,1.005268,1.006742,0.987532,1.007849,1.179784


In [ ]:
# ETFs
# WisdomTree India Earnings Fund (EPI)
# NYSEArca - Nasdaq Real Time Price. Currency in USD
# WEB: https://finance.yahoo.com/quote/EPI/history?p=EPI
epi_etf_daily = yf.download(
    tickers = "EPI",
    period = "max",
    interval = "1d"
)

epi_etf_daily.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-05,49.32,49.349998,49.139999,49.349998,49.349998,978200


In [ ]:

epi_etf_daily_to_merge = get_growth_df(epi_etf_daily, 'epi')
epi_etf_daily_to_merge.tail(2)

,growth_epi_1d,growth_epi_3d,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d
Date,,,,,,
2024-07-03,1.006149,1.016777,1.026129,1.071132,1.103890,1.460117
2024-07-05,1.005296,1.015432,1.033075,1.070267,1.115506,1.460047


In [ ]:
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPPOT
gdppot = pdr.DataReader("GDPPOT", "fred", start=start)
gdppot['gdppot_us_yoy'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(4)-1
gdppot['gdppot_us_qoq'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(1)-1
gdppot.tail(2)


,GDPPOT,gdppot_us_yoy,gdppot_us_qoq
DATE,,,
2024-04-01,22682.66,0.020474,0.005143
2024-07-01,22800.63,0.020675,0.005201


In [ ]:
gdppot_to_merge = gdppot[['gdppot_us_yoy','gdppot_us_qoq']]
gdppot_to_merge.tail(1)

,gdppot_us_yoy,gdppot_us_qoq
DATE,,
2024-07-01,0.020675,0.005201


In [ ]:
# # "Core CPI index", MONTHLY
# https://fred.stlouisfed.org/series/CPILFESL
# The "Consumer Price Index for All Urban Consumers: All Items Less Food & Energy"
# is an aggregate of prices paid by urban consumers for a typical basket of goods, excluding food and energy.
# This measurement, known as "Core CPI," is widely used by economists because food and energy have very volatile prices.
cpilfesl = pdr.DataReader("CPILFESL", "fred", start=start)
cpilfesl['cpi_core_yoy'] = cpilfesl.CPILFESL/cpilfesl.CPILFESL.shift(12)-1
cpilfesl['cpi_core_mom'] = cpilfesl.CPILFESL/cpilfesl.CPILFESL.shift(1)-1

cpilfesl.tail(2)

,CPILFESL,cpi_core_yoy,cpi_core_mom
DATE,,,
2024-04-01,317.622,0.036155,0.002918
2024-05-01,318.140,0.034111,0.001631


In [ ]:
cpilfesl_to_merge = cpilfesl[['cpi_core_yoy','cpi_core_mom']]
cpilfesl_to_merge.tail(2)

,cpi_core_yoy,cpi_core_mom
DATE,,
2024-04-01,0.036155,0.002918
2024-05-01,0.034111,0.001631


In [ ]:
# Fed rate https://fred.stlouisfed.org/series/FEDFUNDS
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start=start)
fedfunds.tail(2)


,FEDFUNDS
DATE,
2024-05-01,5.33
2024-06-01,5.33


In [ ]:
# https://fred.stlouisfed.org/series/DGS1
dgs1 = pdr.DataReader("DGS1", "fred", start=start)
dgs1.tail(2)

,DGS1
DATE,
2024-07-02,5.07
2024-07-03,5.04


In [ ]:
# https://fred.stlouisfed.org/series/DGS5
dgs5 = pdr.DataReader("DGS5", "fred", start=start)
dgs5.tail(2)

,DGS5
DATE,
2024-07-02,4.39
2024-07-03,4.33


In [ ]:
# https://fred.stlouisfed.org/series/DGS10
dgs10 = pdr.DataReader("DGS10", "fred", start=start)
dgs10.tail(2)

,DGS10
DATE,
2024-07-02,4.43
2024-07-03,4.36


VIX - Volatility Index

In [ ]:
# VIX - Volatility Index
# https://finance.yahoo.com/quote/%5EVIX/
vix = yf.download(
    tickers = "^VIX",
    period = "max",
    interval = "1d"
)

vix.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,12.91,12.91,12.86,12.86,12.86,0


In [ ]:
vix_to_merge = vix['Adj Close']
vix_to_merge.tail()

Date
2024-07-01    12.22
2024-07-02    12.03
2024-07-03    12.09
2024-07-05    12.48
2024-07-08    12.86
Name: Adj Close, dtype: float64

## Other assets

In [ ]:
# GOLD
# WEB: https://finance.yahoo.com/quote/GC%3DF
gold = yf.download(
    tickers = "GC=F",
    period = "max",
    interval = "1d"
)

gold.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,2396.199951,2399.300049,2384.699951,2386.399902,2386.399902,36741


In [ ]:
gold_to_merge = get_growth_df(gold,'gold')
gold_to_merge.tail(2)

,growth_gold_1d,growth_gold_3d,growth_gold_7d,growth_gold_30d,growth_gold_90d,growth_gold_365d
Date,,,,,,
2024-07-05,1.012162,1.026164,1.031037,0.986291,1.177471,1.239427
2024-07-08,0.999121,1.027292,1.037926,0.998828,1.173255,1.233983


In [ ]:
# WTI Crude Oil
# WEB: https://uk.finance.yahoo.com/quote/CL=F/
crude_oil = yf.download(
    tickers = "CL=F",
    period = "max",
    interval = "1d"
)

crude_oil.tail(1)



[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,83.150002,83.32,82.470001,82.650002,82.650002,31116


In [ ]:
crude_oil_to_merge = get_growth_df(crude_oil,'wti_oil')
crude_oil_to_merge.tail(2)

,growth_wti_oil_1d,growth_wti_oil_3d,growth_wti_oil_7d,growth_wti_oil_30d,growth_wti_oil_90d,growth_wti_oil_365d
Date,,,,,,
2024-07-05,0.991416,0.997362,1.028826,1.049205,1.071926,1.018868
2024-07-08,0.993867,0.998068,1.021632,1.065489,1.047927,1.031449


In [ ]:
# Brent Oil
# WEB: https://uk.finance.yahoo.com/quote/BZ=F/
brent_oil = yf.download(tickers = "BZ=F",
                     period = "max",
                     interval = "1d")

brent_oil.tail(2)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-05,87.029999,87.949997,86.489998,86.540001,86.540001,39021
2024-07-08,86.750000,86.910004,85.980003,86.139999,86.139999,4597


In [ ]:
brent_oil_to_merge = get_growth_df(brent_oil,'brent_oil')
brent_oil_to_merge.tail(2)

,growth_brent_oil_1d,growth_brent_oil_3d,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d
Date,,,,,,
2024-07-05,0.990840,0.999307,1.017998,1.04416,1.048588,0.981290
2024-07-08,0.995378,0.998840,1.010440,1.05177,1.029767,1.000116


In [ ]:
# https://finance.yahoo.com/quote/BTC-USD/
btc_usd =  yf.download(tickers = "BTC-USD",
                     period = "max",
                     interval = "1d")

btc_usd.tail(2)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-07,58239.429688,58371.121094,55793.324219,55849.109375,55849.109375,20553359505
2024-07-08,55849.574219,55990.632812,54424.457031,55843.195312,55843.195312,26270877696


In [ ]:
btc_usd_to_merge = get_growth_df(btc_usd,'btc_usd')
btc_usd_to_merge.tail(2)

,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
Date,,,,,,
2024-07-07,0.957903,0.980192,0.891044,0.805409,0.779674,1.843659
2024-07-08,0.999894,0.985543,0.888487,0.805751,0.807694,1.850875


## Technical indicators

In [ ]:
def talib_get_momentum_indicators_for_one_ticker(df: pd.DataFrame) -> pd.DataFrame:
    """
    https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/momentum_indicators.md
    """
    # ADX - Average Directional Movement Index
    talib_momentum_adx = talib.ADX(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    # ADXR - Average Directional Movement Index Rating
    talib_momentum_adxr = talib.ADXR(df.High.values, df.Low.values, df.Close.values, timeperiod=14 )
    # APO - Absolute Price Oscillator
    talib_momentum_apo = talib.APO(df.Close.values, fastperiod=12, slowperiod=26, matype=0 )
    # AROON - Aroon
    talib_momentum_aroon = talib.AROON(df.High.values, df.Low.values, timeperiod=14 )
    # talib_momentum_aroon[0].size
    # talib_momentum_aroon[1].size
    # AROONOSC - Aroon Oscillator
    talib_momentum_aroonosc = talib.AROONOSC(df.High.values, df.Low.values, timeperiod=14)
    # BOP - Balance of Power
    # https://school.stockcharts.com/doku.php?id=technical_indicators:balance_of_power
      #calculate open prices as shifted closed prices from the prev day
      # open = df.Last.shift(1)
    talib_momentum_bop = talib.BOP(df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CCI - Commodity Channel Index
    talib_momentum_cci = talib.CCI(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    # CMO - Chande Momentum Oscillator
    talib_momentum_cmo = talib.CMO(df.Close.values, timeperiod=14)
    # DX - Directional Movement Index
    talib_momentum_dx = talib.DX(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    # MACD - Moving Average Convergence/Divergence
    talib_momentum_macd, talib_momentum_macdsignal, talib_momentum_macdhist = talib.MACD(
        df.Close.values, fastperiod=12, slowperiod=26, signalperiod=9
    )
    # MACDEXT - MACD with controllable MA type
    talib_momentum_macd_ext, talib_momentum_macdsignal_ext, talib_momentum_macdhist_ext = talib.MACDEXT(
        df.Close.values, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0
    )
    # MACDFIX - Moving Average Convergence/Divergence Fix 12/26
    talib_momentum_macd_fix, talib_momentum_macdsignal_fix, talib_momentum_macdhist_fix = talib.MACDFIX(
        df.Close.values, signalperiod=9
    )
    # MFI - Money Flow Index
    talib_momentum_mfi = talib.MFI(df.High.values, df.Low.values, df.Close.values, df.Volume.values, timeperiod=14)
    # MINUS_DI - Minus Directional Indicator
    talib_momentum_minus_di = talib.MINUS_DM(df.High.values, df.Low.values, timeperiod=14)
    # MOM - Momentum
    talib_momentum_mom = talib.MOM(df.Close.values, timeperiod=10)
    # PLUS_DI - Plus Directional Indicator
    talib_momentum_plus_di = talib.PLUS_DI(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    # PLUS_DM - Plus Directional Movement
    talib_momentum_plus_dm = talib.PLUS_DM(df.High.values, df.Low.values, timeperiod=14)
    # PPO - Percentage Price Oscillator
    talib_momentum_ppo = talib.PPO(df.Close.values, fastperiod=12, slowperiod=26, matype=0)
    # ROC - Rate of change : ((price/prevPrice)-1)*100
    talib_momentum_roc = talib.ROC(df.Close.values, timeperiod=10)
    # ROCP - Rate of change Percentage: (price-prevPrice)/prevPrice
    talib_momentum_rocp = talib.ROCP(df.Close.values, timeperiod=10)
    # ROCR - Rate of change ratio: (price/prevPrice)
    talib_momentum_rocr = talib.ROCR(df.Close.values, timeperiod=10)
    # ROCR100 - Rate of change ratio 100 scale: (price/prevPrice)*100
    talib_momentum_rocr100 = talib.ROCR100(df.Close.values, timeperiod=10)
    # RSI - Relative Strength Index
    talib_momentum_rsi = talib.RSI(df.Close.values, timeperiod=14)
    # STOCH - Stochastic
    talib_momentum_slowk, talib_momentum_slowd = talib.STOCH(
        df.High.values, df.Low.values, df.Close.values, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0
    )
    # STOCHF - Stochastic Fast
    talib_momentum_fastk, talib_momentum_fastd = talib.STOCHF(
        df.High.values, df.Low.values, df.Close.values, fastk_period=5, fastd_period=3, fastd_matype=0
    )
    # STOCHRSI - Stochastic Relative Strength Index
    talib_momentum_fastk_rsi, talib_momentum_fastd_rsi = talib.STOCHRSI(
        df.Close.values, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0
    )
    # TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    talib_momentum_trix = talib.TRIX(df.Close.values, timeperiod=30)
    # ULTOSC - Ultimate Oscillator
    talib_momentum_ultosc = talib.ULTOSC(df.High.values, df.Low.values, df.Close.values, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    # WILLR - Williams' %R
    talib_momentum_willr = talib.WILLR(df.High.values, df.Low.values, df.Close.values, timeperiod=14)

    momentum_df =   pd.DataFrame({
        # assume here multi-index
        # 'datetime': df.index.get_level_values(0),
        # 'ticker': df.index.get_level_values(1) ,

        # old way with separate columns
        'Date': df.Date.values,
        'Ticker': df.Ticker,

        'adx': talib_momentum_adx,
        'adxr': talib_momentum_adxr,
        'apo': talib_momentum_apo,
        'aroon_1': talib_momentum_aroon[0] ,
        'aroon_2': talib_momentum_aroon[1],
        'aroonosc': talib_momentum_aroonosc,
        'bop': talib_momentum_bop,
        'cci': talib_momentum_cci,
        'cmo': talib_momentum_cmo,
        'dx': talib_momentum_dx,
        'macd': talib_momentum_macd,
        'macdsignal': talib_momentum_macdsignal,
        'macdhist': talib_momentum_macdhist,
        'macd_ext': talib_momentum_macd_ext,
        'macdsignal_ext': talib_momentum_macdsignal_ext,
        'macdhist_ext': talib_momentum_macdhist_ext,
        'macd_fix': talib_momentum_macd_fix,
        'macdsignal_fix': talib_momentum_macdsignal_fix,
        'macdhist_fix': talib_momentum_macdhist_fix,
        'mfi': talib_momentum_mfi,
        'minus_di': talib_momentum_minus_di,
        'mom': talib_momentum_mom,
        'plus_di': talib_momentum_plus_di,
        'dm': talib_momentum_plus_dm,
        'ppo': talib_momentum_ppo,
        'roc': talib_momentum_roc,
        'rocp': talib_momentum_rocp,
        'rocr': talib_momentum_rocr,
        'rocr100': talib_momentum_rocr100,
        'rsi': talib_momentum_rsi,
        'slowk': talib_momentum_slowk,
        'slowd': talib_momentum_slowd,
        'fastk': talib_momentum_fastk,
        'fastd': talib_momentum_fastd,
        'fastk_rsi': talib_momentum_fastk_rsi,
        'fastd_rsi': talib_momentum_fastd_rsi,
        'trix': talib_momentum_trix,
        'ultosc': talib_momentum_ultosc,
        'willr': talib_momentum_willr,
    })
    return momentum_df


In [ ]:
def talib_get_volume_volatility_cycle_price_indicators(df: pd.DataFrame) -> pd.DataFrame:
    # TA-Lib Volume indicators
    # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/volume_indicators.md
    # AD - Chaikin A/D Line
    talib_ad = talib.AD(
        df.High.values, df.Low.values, df.Close.values, df.Volume.values)
    # ADOSC - Chaikin A/D Oscillator
    talib_adosc = talib.ADOSC(
        df.High.values, df.Low.values, df.Close.values, df.Volume.values, fastperiod=3, slowperiod=10)
    # OBV - On Balance Volume
    talib_obv = talib.OBV(
        df.Close.values, df.Volume.values)

    # TA-Lib Volatility indicators
    # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/volatility_indicators.md
    # ATR - Average True Range
    talib_atr = talib.ATR(
        df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    # NATR - Normalized Average True Range
    talib_natr = talib.NATR(
        df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    # OBV - On Balance Volume
    talib_obv = talib.OBV(
        df.Close.values, df.Volume.values)

    # TA-Lib Cycle Indicators
    # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/cycle_indicators.md
    # HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
    talib_ht_dcperiod = talib.HT_DCPERIOD(df.Close.values)
    # HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
    talib_ht_dcphase = talib.HT_DCPHASE(df.Close.values)
    # HT_PHASOR - Hilbert Transform - Phasor Components
    talib_ht_phasor_inphase, talib_ht_phasor_quadrature = talib.HT_PHASOR(
        df.Close.values)
    # HT_SINE - Hilbert Transform - SineWave
    talib_ht_sine_sine, talib_ht_sine_leadsine = talib.HT_SINE(
        df.Close.values)
    # HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode
    talib_ht_trendmode = talib.HT_TRENDMODE(df.Close.values)

    # TA-Lib Price Transform Functions
    # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/price_transform.md
    # AVGPRICE - Average Price
    talib_avgprice = talib.AVGPRICE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # MEDPRICE - Median Price
    talib_medprice = talib.MEDPRICE(df.High.values, df.Low.values)
    # TYPPRICE - Typical Price
    talib_typprice = talib.TYPPRICE(
        df.High.values, df.Low.values, df.Close.values)
    # WCLPRICE - Weighted Close Price
    talib_wclprice = talib.WCLPRICE(
        df.High.values, df.Low.values, df.Close.values)

    volume_volatility_cycle_price_df = pd.DataFrame(
        {'Date': df.Date.values,
          'Ticker': df.Ticker,
          # TA-Lib Volume indicators
          'ad': talib_ad,
          'adosc': talib_adosc,
          'obv': talib_obv,
          # TA-Lib Volatility indicators
          'atr': talib_atr,
          'natr': talib_natr,
          'obv': talib_obv,
          # TA-Lib Cycle Indicators
          'ht_dcperiod': talib_ht_dcperiod,
          'ht_dcphase': talib_ht_dcphase,
          'ht_phasor_inphase': talib_ht_phasor_inphase,
          'ht_phasor_quadrature': talib_ht_phasor_quadrature,
          'ht_sine_sine': talib_ht_sine_sine,
          'ht_sine_leadsine': talib_ht_sine_leadsine,
          'ht_trendmod': talib_ht_trendmode,
          # TA-Lib Price Transform Functions
          'avgprice': talib_avgprice,
          'medprice': talib_medprice,
          'typprice': talib_typprice,
          'wclprice': talib_wclprice,
          }
    )

    # Need a proper date type
    volume_volatility_cycle_price_df['Date'] = pd.to_datetime(
        volume_volatility_cycle_price_df['Date'])

    return volume_volatility_cycle_price_df

In [ ]:
def talib_get_pattern_recognition_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """ TA-Lib Pattern Recognition indicators
        # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/pattern_recognition.md
        # Nice article about candles (pattern recognition) https://medium.com/analytics-vidhya/recognizing-over-50-candlestick-patterns-with-python-4f02a1822cb5
    """
    # CDL2CROWS - Two Crows
    talib_cdl2crows = talib.CDL2CROWS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDL3BLACKCROWS - Three Black Crows
    talib_cdl3blackrows = talib.CDL3BLACKCROWS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDL3INSIDE - Three Inside Up/Down
    talib_cdl3inside = talib.CDL3INSIDE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDL3LINESTRIKE - Three-Line Strike
    talib_cdl3linestrike = talib.CDL3LINESTRIKE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDL3OUTSIDE - Three Outside Up/Down
    talib_cdl3outside = talib.CDL3OUTSIDE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDL3STARSINSOUTH - Three Stars In The South
    talib_cdl3starsinsouth = talib.CDL3STARSINSOUTH(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDL3WHITESOLDIERS - Three Advancing White Soldiers
    talib_cdl3whitesoldiers = talib.CDL3WHITESOLDIERS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLABANDONEDBABY - Abandoned Baby
    talib_cdlabandonedbaby = talib.CDLABANDONEDBABY(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLADVANCEBLOCK - Advance Block
    talib_cdladvancedblock = talib.CDLADVANCEBLOCK(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLBELTHOLD - Belt-hold
    talib_cdlbelthold = talib.CDLBELTHOLD(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLBREAKAWAY - Breakaway
    talib_cdlbreakaway = talib.CDLBREAKAWAY(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLCLOSINGMARUBOZU - Closing Marubozu
    talib_cdlclosingmarubozu = talib.CDLCLOSINGMARUBOZU(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLCONCEALBABYSWALL - Concealing Baby Swallow
    talib_cdlconcealbabyswall = talib.CDLCONCEALBABYSWALL(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLCOUNTERATTACK - Counterattack
    talib_cdlcounterattack = talib.CDLCOUNTERATTACK(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLDARKCLOUDCOVER - Dark Cloud Cover
    talib_cdldarkcloudcover = talib.CDLDARKCLOUDCOVER(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLDOJI - Doji
    talib_cdldoji = talib.CDLDOJI(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLDOJISTAR - Doji Star
    talib_cdldojistar = talib.CDLDOJISTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLDRAGONFLYDOJI - Dragonfly Doji
    talib_cdldragonflydoji = talib.CDLDRAGONFLYDOJI(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLENGULFING - Engulfing Pattern
    talib_cdlengulfing = talib.CDLENGULFING(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)

    # CDLEVENINGDOJISTAR - Evening Doji Star
    talib_cdleveningdojistar = talib.CDLEVENINGDOJISTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLEVENINGSTAR - Evening Star
    talib_cdleveningstar = talib.CDLEVENINGSTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLGAPSIDESIDEWHITE - Up/Down-gap side-by-side white lines
    talib_cdlgapsidesidewhite = talib.CDLGAPSIDESIDEWHITE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLGRAVESTONEDOJI - Gravestone Doji
    talib_cdlgravestonedoji = talib.CDLGRAVESTONEDOJI(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHAMMER - Hammer
    talib_cdlhammer = talib.CDLHAMMER(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHANGINGMAN - Hanging Man
    talib_cdlhangingman = talib.CDLHANGINGMAN(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHARAMI - Harami Pattern
    talib_cdlharami = talib.CDLHARAMI(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHARAMICROSS - Harami Cross Pattern
    talib_cdlharamicross = talib.CDLHARAMICROSS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHIGHWAVE - High-Wave Candle
    talib_cdlhighwave = talib.CDLHIGHWAVE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHIKKAKE - Hikkake Pattern
    talib_cdlhikkake = talib.CDLHIKKAKE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLHIKKAKEMOD - Modified Hikkake Pattern
    talib_cdlhikkakemod = talib.CDLHIKKAKEMOD(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)

    # CDLHOMINGPIGEON - Homing Pigeon
    talib_cdlhomingpigeon = talib.CDLHOMINGPIGEON(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLIDENTICAL3CROWS - Identical Three Crows
    talib_cdlidentical3crows = talib.CDLIDENTICAL3CROWS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLINNECK - In-Neck Pattern
    talib_cdlinneck = talib.CDLINNECK(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLINVERTEDHAMMER - Inverted Hammer
    talib_cdlinvertedhammer = talib.CDLINVERTEDHAMMER(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLKICKING - Kicking
    talib_cdlkicking = talib.CDLKICKING(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLKICKINGBYLENGTH - Kicking - bull/bear determined by the longer marubozu
    talib_cdlkickingbylength = talib.CDLKICKINGBYLENGTH(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLLADDERBOTTOM - Ladder Bottom
    talib_cdlladderbottom = talib.CDLLADDERBOTTOM(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLLONGLEGGEDDOJI - Long Legged Doji
    talib_cdllongleggeddoji = talib.CDLLONGLEGGEDDOJI(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLLONGLINE - Long Line Candle
    talib_cdllongline = talib.CDLLONGLINE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLMARUBOZU - Marubozu
    talib_cdlmarubozu = talib.CDLMARUBOZU(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLMATCHINGLOW - Matching Low
    talib_cdlmatchinglow = talib.CDLMATCHINGLOW(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)

    # CDLMATHOLD - Mat Hold
    talib_cdlmathold = talib.CDLMATHOLD(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLMORNINGDOJISTAR - Morning Doji Star
    talib_cdlmorningdojistar = talib.CDLMORNINGDOJISTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLMORNINGSTAR - Morning Star
    talib_cdlmorningstar = talib.CDLMORNINGSTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
    # CDLONNECK - On-Neck Pattern
    talib_cdlonneck = talib.CDLONNECK(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLPIERCING - Piercing Pattern
    talib_cdlpiercing = talib.CDLPIERCING(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLRICKSHAWMAN - Rickshaw Man
    talib_cdlrickshawman = talib.CDLRICKSHAWMAN(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLRISEFALL3METHODS - Rising/Falling Three Methods
    talib_cdlrisefall3methods = talib.CDLRISEFALL3METHODS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLSEPARATINGLINES - Separating Lines
    talib_cdlseparatinglines = talib.CDLSEPARATINGLINES(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLSHOOTINGSTAR - Shooting Star
    talib_cdlshootingstar = talib.CDLSHOOTINGSTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLSHORTLINE - Short Line Candle
    talib_cdlshortline = talib.CDLSHORTLINE(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLSPINNINGTOP - Spinning Top
    talib_cdlspinningtop = talib.CDLSPINNINGTOP(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)

    # CDLSTALLEDPATTERN - Stalled Pattern
    talib_cdlstalledpattern = talib.CDLSTALLEDPATTERN(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLSTICKSANDWICH - Stick Sandwich
    talib_cdlsticksandwich = talib.CDLSTICKSANDWICH(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLTAKURI - Takuri (Dragonfly Doji with very long lower shadow)
    talib_cdltakuru = talib.CDLTAKURI(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLTASUKIGAP - Tasuki Gap
    talib_cdltasukigap = talib.CDLTASUKIGAP(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLTHRUSTING - Thrusting Pattern
    talib_cdlthrusting = talib.CDLTHRUSTING(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLTRISTAR - Tristar Pattern
    talib_cdltristar = talib.CDLTRISTAR(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLUNIQUE3RIVER - Unique 3 River
    talib_cdlunique3river = talib.CDLUNIQUE3RIVER(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLUPSIDEGAP2CROWS - Upside Gap Two Crows
    talib_cdlupsidegap2crows = talib.CDLUPSIDEGAP2CROWS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)
    # CDLXSIDEGAP3METHODS - Upside/Downside Gap Three Methods
    talib_cdlxsidegap3methods = talib.CDLXSIDEGAP3METHODS(
        df.Open.values, df.High.values, df.Low.values, df.Close.values)

    pattern_indicators_df = pd.DataFrame(
        {'Date': df.Date.values,
        'Ticker': df.Ticker,
        # TA-Lib Pattern Recognition indicators
        'cdl2crows': talib_cdl2crows,
        'cdl3blackrows': talib_cdl3blackrows,
        'cdl3inside': talib_cdl3inside,
        'cdl3linestrike': talib_cdl3linestrike,
        'cdl3outside': talib_cdl3outside,
        'cdl3starsinsouth': talib_cdl3starsinsouth,
        'cdl3whitesoldiers': talib_cdl3whitesoldiers,
        'cdlabandonedbaby': talib_cdlabandonedbaby,
        'cdladvancedblock': talib_cdladvancedblock,
        'cdlbelthold': talib_cdlbelthold,
        'cdlbreakaway': talib_cdlbreakaway,
        'cdlclosingmarubozu': talib_cdlclosingmarubozu,
        'cdlconcealbabyswall': talib_cdlconcealbabyswall,
        'cdlcounterattack': talib_cdlcounterattack,
        'cdldarkcloudcover': talib_cdldarkcloudcover,
        'cdldoji': talib_cdldoji,
        'cdldojistar': talib_cdldojistar,
        'cdldragonflydoji': talib_cdldragonflydoji,
        'cdlengulfing': talib_cdlengulfing,
        'cdleveningdojistar': talib_cdleveningdojistar,
        'cdleveningstar': talib_cdleveningstar,
        'cdlgapsidesidewhite': talib_cdlgapsidesidewhite,
        'cdlgravestonedoji': talib_cdlgravestonedoji,
        'cdlhammer': talib_cdlhammer,
        'cdlhangingman': talib_cdlhangingman,
        'cdlharami': talib_cdlharami,
        'cdlharamicross': talib_cdlharamicross,
        'cdlhighwave': talib_cdlhighwave,
        'cdlhikkake': talib_cdlhikkake,
        'cdlhikkakemod': talib_cdlhikkakemod,
        'cdlhomingpigeon': talib_cdlhomingpigeon,
        'cdlidentical3crows': talib_cdlidentical3crows,
        'cdlinneck': talib_cdlinneck,
        'cdlinvertedhammer': talib_cdlinvertedhammer,
        'cdlkicking': talib_cdlkicking,
        'cdlkickingbylength': talib_cdlkickingbylength,
        'cdlladderbottom': talib_cdlladderbottom,
        'cdllongleggeddoji': talib_cdllongleggeddoji,
        'cdllongline': talib_cdllongline,
        'cdlmarubozu': talib_cdlmarubozu,
        'cdlmatchinglow': talib_cdlmatchinglow,
        'cdlmathold': talib_cdlmathold,
        'cdlmorningdojistar': talib_cdlmorningdojistar,
        'cdlmorningstar': talib_cdlmorningstar,
        'cdlonneck': talib_cdlonneck,
        'cdlpiercing': talib_cdlpiercing,
        'cdlrickshawman': talib_cdlrickshawman,
        'cdlrisefall3methods': talib_cdlrisefall3methods,
        'cdlseparatinglines': talib_cdlseparatinglines,
        'cdlshootingstar': talib_cdlshootingstar,
        'cdlshortline': talib_cdlshortline,
        'cdlspinningtop': talib_cdlspinningtop,
        'cdlstalledpattern': talib_cdlstalledpattern,
        'cdlsticksandwich': talib_cdlsticksandwich,
        'cdltakuru': talib_cdltakuru,
        'cdltasukigap': talib_cdltasukigap,
        'cdlthrusting': talib_cdlthrusting,
        'cdltristar': talib_cdltristar,
        'cdlunique3river': talib_cdlunique3river,
        'cdlupsidegap2crows': talib_cdlupsidegap2crows,
        'cdlxsidegap3methods': talib_cdlxsidegap3methods
    })

    # Need a proper date type
    pattern_indicators_df['Date'] = pd.to_datetime(
        pattern_indicators_df['Date']
    )

    return pattern_indicators_df

Scope of stocks

In [ ]:

# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

# https://companiesmarketcap.com/china/largest-companies-in-china-by-market-cap/
CHINA_STOCKS = ['TCEHY','1398.HK','601857.SS','600519.SS','0941.HK','601288.SS','PDD','BABA','601939.SS','601988.SS']

ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS + CHINA_STOCKS

In [ ]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

40 BABA


41 601939.SS


[*********************100%%**********************]  1 of 1 completed


42 601988.SS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker: str, us_stocks_list: list, eu_stocks_list: list, india_stocks_list: list, china_stocks_list: list) -> str:
    if ticker in us_stocks_list:
        return 'US'
    elif ticker in eu_stocks_list:
        return 'EU'
    elif ticker in india_stocks_list:
        return 'INDIA'
    elif ticker in china_stocks_list:
        return 'CHINA'
    else:
        return 'ERROR'

In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS, CHINA_STOCKS))

# count of observations between US-EU-INDIA-CHINA stocks
stocks_df.ticker_type.value_counts()


ticker_type
US       81168
EU       77529
INDIA    63801
CHINA    39620
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

43

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13135
JPM              11169
AAPL             10981
NVO              10886
MSFT              9655
CDI.PA            8372
TTE               8233
ASML              7378
SAP               7249
HINDUNILVR.NS     7164
SBIN.NS           7162
RELIANCE.NS       7161
ITC.NS            7161
SIE.DE            7090
BRK-B             7086
AMZN              6829
NVDA              6404
INFY              6371
RMS.PA            6302
MC.PA             6302
OR.PA             6302
0941.HK           6115
IBN               6106
ACN               5776
HDB               5775
600519.SS         5633
LT.NS             5469
BHARTIARTL.NS     5466
TCS.NS            5437
GOOG              5003
601988.SS         4380
1398.HK           4360
V                 4102
601939.SS         4077
601857.SS         4053
AVGO              3753
TCEHY             3649
IDEXY             3639
601288.SS         3394
META              3051
BABA              2464
PDD               1495
LICI.NS            529
Name

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
0941.HK,CHINA,2000-01-04,2024-07-08
1398.HK,CHINA,2006-10-27,2024-07-08
600519.SS,CHINA,2001-08-27,2024-07-08
601288.SS,CHINA,2010-07-15,2024-07-08
601857.SS,CHINA,2007-11-05,2024-07-08
601939.SS,CHINA,2007-09-25,2024-07-08
601988.SS,CHINA,2006-07-05,2024-07-08
AAPL,US,1980-12-12,2024-07-05
ACN,EU,2001-07-19,2024-07-05


In [ ]:
# average growth 365days after 2020
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  CHINA          1.374680
      EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  CHINA          1.330600
      EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  CHINA          0.878291
      EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  CHINA          1.169413
      EU             1.247590
      INDIA          1.201855
      US             1.203253
2024  CHINA          1.418766
      EU             1.577120
      INDIA          1.299694
      US             2.274506
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()


<class 'pandas.core.frame.DataFrame'>
Index: 223651 entries, 3490 to 262117
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          223651 non-null  float64       
 1   High                          223651 non-null  float64       
 2   Low                           223651 non-null  float64       
 3   Close                         223651 non-null  float64       
 4   Adj Close                     223651 non-null  float64       
 5   Volume                        223651 non-null  int64         
 6   Ticker                        223651 non-null  object        
 7   Year                          223651 non-null  int32         
 8   Month                         223651 non-null  int32         
 9   Weekday                       223651 non-null  int32         
 10  Date                          223651 non-null  datetime64[ns]
 11  growth_1d      

### Collect tech indicators

In [ ]:
# Volume needs to be float, not int
stocks_df['Volume'] = stocks_df['Volume'] * 1.0

# to resolve an error "Exception: input array type is not double"
# https://stackoverflow.com/questions/51712269/how-to-run-ta-lib-on-multiple-columns-of-a-pandas-dataframe
for f in ['Open','High','Low','Close', 'Volume', 'Adj Close']:
    stocks_df.loc[:,f] = stocks_df.loc[:,f].astype('float64')

stocks_df.info()


# adding Momentum / Pattern/ Volume features to all tickers - one by one
merged_df_with_tech_ind = pd.DataFrame({'A' : []})

current_ticker_data = None
i = 0
for ticker in ALL_TICKERS:
    i += 1
    print(f'{i}/{len(ALL_TICKERS)} Current ticker is {ticker}')
    current_ticker_data = stocks_df[stocks_df.Ticker.isin([ticker])]
    # need to have same 'utc' time on both sides
    # https://stackoverflow.com/questions/73964894/you-are-trying-to-merge-on-datetime64ns-utc-and-datetime64ns-columns-if-yo
    current_ticker_data['Date']= pd.to_datetime(current_ticker_data['Date'], utc=True)

    # 3 calls to get additional features
    df_current_ticker_momentum_indicators = talib_get_momentum_indicators_for_one_ticker(current_ticker_data)
    df_current_ticker_momentum_indicators["Date"]= pd.to_datetime(df_current_ticker_momentum_indicators['Date'], utc=True)
    # df_current_ticker_momentum_indicators.loc[:,"Date"]= pd.to_datetime(df_current_ticker_momentum_indicators['Date'], utc=True)

    df_current_ticker_volume_indicators = talib_get_volume_volatility_cycle_price_indicators(current_ticker_data)
    df_current_ticker_volume_indicators["Date"]= pd.to_datetime(df_current_ticker_volume_indicators['Date'], utc=True)
    # df_current_ticker_volume_indicators.loc[:,"Date"]= pd.to_datetime(df_current_ticker_volume_indicators['Date'], utc=True)

    df_current_ticker_pattern_indicators = talib_get_pattern_recognition_indicators(current_ticker_data)
    df_current_ticker_pattern_indicators["Date"]= pd.to_datetime(df_current_ticker_pattern_indicators['Date'], utc=True)
    # df_current_ticker_pattern_indicators.loc[:,"Date"]= pd.to_datetime(df_current_ticker_pattern_indicators['Date'], utc=True)

    # merge to one df
    m1 = pd.merge(current_ticker_data, df_current_ticker_momentum_indicators.reset_index(), how = 'left', on = ["Date","Ticker"], validate = "one_to_one")
    m2 = pd.merge(m1, df_current_ticker_volume_indicators.reset_index(), how = 'left', on = ["Date","Ticker"], validate = "one_to_one")
    m3 = pd.merge(m2, df_current_ticker_pattern_indicators.reset_index(), how = 'left', on = ["Date","Ticker"], validate = "one_to_one")

    if merged_df_with_tech_ind.empty:
        merged_df_with_tech_ind = m3
    else:
        merged_df_with_tech_ind = pd.concat([merged_df_with_tech_ind,m3], ignore_index = False)

    # ensure the datetime column
    merged_df_with_tech_ind['Date'] = pd.to_datetime(merged_df_with_tech_ind['Date'], utc=True).dt.tz_localize(None)


print(merged_df_with_tech_ind.info())
merged_df_with_tech_ind[merged_df_with_tech_ind.Date=='2024-04-01'].tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262118 entries, 0 to 262117
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          262118 non-null  float64       
 1   High                          262118 non-null  float64       
 2   Low                           262118 non-null  float64       
 3   Close                         262118 non-null  float64       
 4   Adj Close                     262118 non-null  float64       
 5   Volume                        262118 non-null  float64       
 6   Ticker                        262118 non-null  object        
 7   Year                          262118 non-null  int32         
 8   Month                         262118 non-null  int32         
 9   Weekday                       262118 non-null  int32         
 10  Date                          262118 non-null  datetime64[ns]
 11  growth_1d    

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,cdlspinningtop,cdlstalledpattern,cdlsticksandwich,cdltakuru,cdltasukigap,cdlthrusting,cdltristar,cdlunique3river,cdlupsidegap2crows,cdlxsidegap3methods
3329,4.230000,4.250000,4.210000,4.230000,4.009524,253839301.0,601288.SS,2024,4,0,...,100,0,0,0,0,0,0,0,0,0
1428,117.900002,121.360001,117.230003,119.160004,119.160004,6163300.0,PDD,2024,4,0,...,0,0,0,0,0,0,0,0,0,0
2397,73.239998,73.582001,72.809998,73.370003,71.809334,14311700.0,BABA,2024,4,0,...,100,0,0,0,0,0,0,0,0,0
4012,6.860000,6.920000,6.850000,6.910000,6.910000,69844058.0,601939.SS,2024,4,0,...,0,0,0,0,0,0,0,0,0,0
4315,4.410000,4.470000,4.390000,4.460000,4.460000,180010859.0,601988.SS,2024,4,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# merge with DAX daily

m2 = pd.merge(
    merged_df_with_tech_ind,
    dax_daily_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

print(m2.shape)
m2.tail(1)

(262118, 150)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,cdltristar,cdlunique3river,cdlupsidegap2crows,cdlxsidegap3methods,growth_dax_1d,growth_dax_3d,growth_dax_7d,growth_dax_30d,growth_dax_90d,growth_dax_365d
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,7,0,...,0,0,0,0,1.003053,1.008562,1.017644,0.987065,1.052874,1.224982


In [ ]:
# merge with SNP500

m3 = pd.merge(
    m2,
    snp500_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

print(m3.shape)
m3.tail(1)


(262118, 156)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_dax_7d,growth_dax_30d,growth_dax_90d,growth_dax_365d,growth_snp500_1d,growth_snp500_3d,growth_snp500_7d,growth_snp500_30d,growth_snp500_90d,growth_snp500_365d
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,7,0,...,1.017644,0.987065,1.052874,1.224982,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# merge with Dow Jones
m4 = pd.merge(
    m3,
    dji_daily_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

print(m4.shape)
m4.tail()

(262118, 162)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_snp500_7d,growth_snp500_30d,growth_snp500_90d,growth_snp500_365d,growth_dji_1d,growth_dji_3d,growth_dji_7d,growth_dji_30d,growth_dji_90d,growth_dji_365d
4375,4.67,4.76,4.66,4.76,4.76,224677163.0,601988.SS,2024,7,1,...,1.008123,1.038795,1.082952,1.402190,1.004144,1.004284,1.004637,0.983208,1.006725,1.181244
4376,4.76,4.79,4.67,4.71,4.71,215707951.0,601988.SS,2024,7,2,...,1.016364,1.043121,1.088080,1.420167,0.999394,1.004835,0.997381,0.987470,1.004510,1.189545
4377,4.71,4.75,4.70,4.74,4.74,146257032.0,601988.SS,2024,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,7,4,...,1.017898,1.046187,1.098167,1.401393,1.001727,1.005268,1.006742,0.987532,1.007849,1.179784
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Merge with WisdomTree India Earnings Fund (EPI)
m5 = pd.merge(
    m4,
    epi_etf_daily_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

print(m5.shape)
m5.tail()


(262118, 168)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_dji_7d,growth_dji_30d,growth_dji_90d,growth_dji_365d,growth_epi_1d,growth_epi_3d,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d
4375,4.67,4.76,4.66,4.76,4.76,224677163.0,601988.SS,2024,7,1,...,1.004637,0.983208,1.006725,1.181244,1.003910,1.011401,1.026726,1.068784,1.096158,1.460735
4376,4.76,4.79,4.67,4.71,4.71,215707951.0,601988.SS,2024,7,2,...,0.997381,0.987470,1.004510,1.189545,1.006149,1.016777,1.026129,1.071132,1.103890,1.460117
4377,4.71,4.75,4.70,4.74,4.74,146257032.0,601988.SS,2024,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,7,4,...,1.006742,0.987532,1.007849,1.179784,1.005296,1.015432,1.033075,1.070267,1.115506,1.460047
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Merge with GDP  - by Quater

# define quarter as the first date of qtr
m5['Quarter'] = m5['Date'].dt.to_period('Q').dt.to_timestamp()
m5['Quarter']

m6 = pd.merge(
    m5,
    gdppot_to_merge,
    how='left',
    left_on='Quarter',
    right_index=True,
    validate = "many_to_one"
)


m6.tail(2)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_dji_365d,growth_epi_1d,growth_epi_3d,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,7,4,...,1.179784,1.005296,1.015432,1.033075,1.070267,1.115506,1.460047,2024-07-01,0.020675,0.005201
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-01,0.020675,0.005201


In [ ]:
# merge with CPI - by month

m6['Month'] = m6['Date'].dt.to_period('M').dt.to_timestamp()
m6['Month']

m7 = pd.merge(
    m6,
    cpilfesl_to_merge,
    how='left',
    left_on='Month',
    right_index=True,
    validate = "many_to_one"
)

# PROBLEM! Last month is not defined
m7.tail(1)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_epi_3d,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,NaN,NaN,NaN,NaN,2024-07-01,0.020675,0.005201,NaN,NaN


In [ ]:
fields_to_fill = ['cpi_core_yoy', 'cpi_core_mom']

# Fill missing values in selected fields with the last defined value
for field in fields_to_fill:
    m7[field] = m7[field].fillna(method='ffill')

m7.tail(1)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_epi_3d,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,NaN,NaN,NaN,NaN,2024-07-01,0.020675,0.005201,0.034111,0.001631


In [ ]:
# merge with fedfunds

m8 = pd.merge(
    m7,
    fedfunds,
    how='left',
    left_on='Month',
    right_index=True,
    validate = "many_to_one"
)

m8.tail(1)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,NaN,NaN,NaN,2024-07-01,0.020675,0.005201,0.034111,0.001631,NaN


In [ ]:
fields_to_fill = ['FEDFUNDS']
# Fill missing values in selected fields with the last defined value
for field in fields_to_fill:
    m8[field] = m8[field].fillna(method='ffill')

In [ ]:
# merge with DGS1

m9 = pd.merge(
    m8,
    dgs1,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

# fix last day later
m9.tail(2)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_epi_30d,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS,DGS1
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.070267,1.115506,1.460047,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,NaN,NaN,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN


In [ ]:
# merge with DGS5

m10 = pd.merge(
    m9,
    dgs5,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

# fix last day later
m10.tail(2)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS,DGS1,DGS5
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.115506,1.460047,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN,NaN
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,NaN,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN,NaN


In [ ]:
# merge with DGS10

m11 = pd.merge(
    m10,
    dgs10,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

m11.tail(2)


,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS,DGS1,DGS5,DGS10
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.460047,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN,NaN,NaN
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN,NaN,NaN


In [ ]:
# merge with VIX

m12 = pd.merge(
    m11,
    vix_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

m12.tail(2)


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS,DGS1,DGS5,DGS10,Adj Close_y
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN,NaN,NaN,12.48
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,2024-07-01,0.020675,0.005201,0.034111,0.001631,5.33,NaN,NaN,NaN,12.86


In [ ]:
# merge with gold

m13 = pd.merge(
    m12,
    gold_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

m13.tail(2)


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,DGS1,DGS5,DGS10,Adj Close_y,growth_gold_1d,growth_gold_3d,growth_gold_7d,growth_gold_30d,growth_gold_90d,growth_gold_365d
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,NaN,NaN,NaN,12.48,1.012162,1.026164,1.031037,0.986291,1.177471,1.239427
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,NaN,NaN,NaN,12.86,0.999121,1.027292,1.037926,0.998828,1.173255,1.233983


In [ ]:
# merge with Crude Oil

m14 = pd.merge(
    m13,
    crude_oil_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

m14.tail(2)


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_gold_7d,growth_gold_30d,growth_gold_90d,growth_gold_365d,growth_wti_oil_1d,growth_wti_oil_3d,growth_wti_oil_7d,growth_wti_oil_30d,growth_wti_oil_90d,growth_wti_oil_365d
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.031037,0.986291,1.177471,1.239427,0.991416,0.997362,1.028826,1.049205,1.071926,1.018868
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,1.037926,0.998828,1.173255,1.233983,0.993867,0.998068,1.021632,1.065489,1.047927,1.031449


In [ ]:
# merge with Brent Oil

m15 = pd.merge(
    m14,
    brent_oil_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
)

m15.tail(2)


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_wti_oil_7d,growth_wti_oil_30d,growth_wti_oil_90d,growth_wti_oil_365d,growth_brent_oil_1d,growth_brent_oil_3d,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.028826,1.049205,1.071926,1.018868,0.990840,0.999307,1.017998,1.04416,1.048588,0.981290
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,1.021632,1.065489,1.047927,1.031449,0.995378,0.998840,1.010440,1.05177,1.029767,1.000116


In [ ]:
# merge with bitcoin

m16 = pd.merge(
    m15,
    btc_usd_to_merge,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
    )

m16.tail(2)



,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.017998,1.04416,1.048588,0.981290,0.994466,0.913482,0.939361,0.797132,0.822432,1.894471
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,1.010440,1.05177,1.029767,1.000116,0.999894,0.985543,0.888487,0.805751,0.807694,1.850875


In [ ]:
# Fill missing values in selected fields with the last defined value
fields_to_fill = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS', 'DGS1', 'DGS5', 'DGS10']

for field in fields_to_fill:
    m16[field] = m16[field].fillna(method='ffill')

m16.tail(2)


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
4378,4.74,4.74,4.74,4.74,4.74,0.0,601988.SS,2024,2024-07-01,4,...,1.017998,1.04416,1.048588,0.981290,0.994466,0.913482,0.939361,0.797132,0.822432,1.894471
4379,4.64,4.71,4.62,4.69,4.69,194208720.0,601988.SS,2024,2024-07-01,0,...,1.010440,1.05177,1.029767,1.000116,0.999894,0.985543,0.888487,0.805751,0.807694,1.850875


In [ ]:
date = m16['Date'].max()
date_str = date.strftime('%Y_%m_%d')
print(date_str)

2024_07_08


In [ ]:
m16.to_parquet(
    f'stocks_df_combined_{date_str}.parquet.brotli',
    compression='brotli'
)